In [22]:
import tensorflow.keras as keras

# Download Shakespeare's dataset
shakespeare_url = "https://homl.info/shakespeare"
file_name = "shakespeare.txt"
filepath = keras.utils.get_file(file_name, shakespeare_url)

# Read/Store text file in string
with open(filepath) as f:
    shakespeare_text = f.read()

# Print a small section of the dataset
print(shakespeare_text[: len(shakespeare_text) // 5000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is 


In [32]:
# Tokenize - encode each CHARACTER as an integer/id
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

# See what the tokenizer looks like -> ex: word - "First"
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]